In [ ]:
import pandas as pd
from diffusers import DiffusionPipeline, KandinskyV22Pipeline, KandinskyV22PriorPipeline
import torch

In [ ]:
pipe_prior = KandinskyV22PriorPipeline.from_pretrained("kandinsky-community/kandinsky-2-2-prior",torch_dtype=torch.float16)
pipe_prior.to("cuda")
pipe_prior.enable_model_cpu_offload()

pipe = KandinskyV22Pipeline.from_pretrained("kandinsky-community/kandinsky-2-2-decoder",torch_dtype=torch.float16)
pipe.to("cuda")
def generate(word):
    prompt = f"{word}, minimalist, watercolor"
    out = pipe_prior(prompt)
    image_emb = out.image_embeds
    zero_image_emb = out.negative_image_embeds
    image = pipe(
        image_embeds=image_emb,
        negative_image_embeds=zero_image_emb,
        height=768,
        width=768,
        num_inference_steps=50,
    ).images
    out.images[0].save(f"{word}.png")
    return

In [ ]:
words = pd.read_csv('100words.csv',sep=';',header=None)

for word in words[0]:
     generate_image(word)